# 2018-10-08 Correct batch effects

Here I'm trying to use the workflow presented as a tutorial to the `scran` R package to do correction of batch effects. There are several steps involved, I'll try to get my head over the whole procedure.

In [ ]:
library(scran)
library(scater)

First we define the name of the files and we load them. Notice that here I put a new `gene_annotations.tsv` file where I saved once and for all all the names of the genes in our list, as extracted by the Ensembl Python API `pyensembl` (release 93).

In [ ]:
# file names
matrices_dir <- "/home/rcortini/work/CRG/projects/sc_hiv/data/matrices"

# P2449
matrix_fname_1 <- sprintf("%s/%s.tsv.gz", matrices_dir, "P2449")
sample_sheet_fname_1 <- sprintf("%s/monocle/%s.pd.tsv", matrices_dir, "P2449")

# P2458
matrix_fname_2 <- sprintf("%s/%s.tsv.gz", matrices_dir, "P2458")
sample_sheet_fname_2 <- sprintf("%s/monocle/%s.pd.tsv", matrices_dir, "P2458")

# gene annotations file
gene_annotations <- sprintf("%s/gene_annotations.tsv", matrices_dir)

In [ ]:
expr_matrix_1 <- read.table(matrix_fname_1, header = TRUE, row.names = 1,
                            sep = "\t", check.names = FALSE)
sample_sheet_1 <- read.delim(sample_sheet_fname_1, header = TRUE, row.names = 1)

expr_matrix_2 <- read.table(matrix_fname_2, header = TRUE, row.names = 1,
                            sep = "\t", check.names = FALSE)
sample_sheet_2 <- read.delim(sample_sheet_fname_2, header = TRUE, row.names = 1)

gene_data <- read.delim(gene_annotations, header = TRUE, row.names = 1, sep = "\t")

Now that we have the data we initialize the basic structure of the experiment by initializing a class called `SingleCellExperiment` which should contain all the information related to a single cell RNA-seq experiment. We supply it with the expression matrices as well as with the sample sheets and the gene annotation file.

In [ ]:
library(SingleCellExperiment)
sce.p2449 <- SingleCellExperiment(list(counts=as.matrix(expr_matrix_1)),
                                  rowData=DataFrame(gene_data),
                                  colData=DataFrame(sample_sheet_1))
sce.p2458 <- SingleCellExperiment(list(counts=as.matrix(expr_matrix_2)),
                                  rowData=DataFrame(gene_data),
                                  colData=DataFrame(sample_sheet_2))

## Quality control
The first downstream step is to do quality control of the experiment. The packages provide us with a function to do exactly that.

In [ ]:
sce.p2449 <- calculateQCMetrics(sce.p2449, compact=TRUE)
sce.p2458 <- calculateQCMetrics(sce.p2458, compact=TRUE)

Once we have the information loaded, we can proceed with removing the outlier (lines of code copied and pasted from the tutorial).

In [ ]:
QC.p2449 <- sce.p2449$scater_qc
low.lib <- isOutlier(QC.p2449$all$log10_total_counts, type="lower", nmad=3)
low.genes <- isOutlier(QC.p2449$all$log10_total_features_by_counts, type="lower",
                      nmad=3)
data.frame(LowLib=sum(low.lib), LowNgenes=sum(low.genes))

In [ ]:
discard.p2449 <- low.lib | low.genes
sce.p2449 <- sce.p2449[,!discard.p2449]
summary(discard.p2449)

In [ ]:
QC.p2458 <- sce.p2458$scater_qc
low.lib <- isOutlier(QC.p2458$all$log10_total_counts, type="lower", nmad=3)
low.genes <- isOutlier(QC.p2458$all$log10_total_features_by_counts, type="lower",
                      nmad=3)
data.frame(LowLib=sum(low.lib), LowNgenes=sum(low.genes))

In [ ]:
discard.p2458 <- low.lib | low.genes
sce.p2458 <- sce.p2458[,!discard.p2458]
summary(discard.p2458)

## Normalization
Once we removed the low-quality cells, we can proceed with the normalization. 

In [ ]:
sizes.p2458 <- c(sum(sce.p2458$label == "Jurkat"),
                 sum(sce.p2458$label == "J-Lat+DMSO"),
                 sum(sce.p2458$label == "J-Lat+SAHA"))
sizes.p2449 <- c(sum(sce.p2449$label == "Jurkat"),
                 sum(sce.p2449$label == "J-Lat+DMSO"),
                 sum(sce.p2449$label == "J-Lat+SAHA"))

In [ ]:
sizefactors.p2449 <- computeSumFactors(counts(sce.p2449), sizes=sizes.p2449)
sizefactors.p2458 <- computeSumFactors(counts(sce.p2458), sizes=sizes.p2458)

In [ ]:
summary(sizefactors.p2449)

In [ ]:
sce.p2449 <- computeSumFactors(sce.p2449, sizes=sizes.p2449)
normalized.p2449 <- normalize(sce.p2449)
sce.p2458 <- computeSumFactors(sce.p2458, sizes=sizes.p2458)
normalized.p2458 <- normalize(sce.p2458)

## Identify highly variable genes

### P2449

In [ ]:
labels <- sort(unique(sce.p2449$label))
par(mfrow=c(ceiling(length(labels)/2), 2), 
    mar=c(4.1, 4.1, 2.1, 0.1))
collected <- list()
for (x in unique(sce.p2449$label)) {
    current <- sce.p2449[,sce.p2449$label==x]
    if (ncol(current)<2L) { next }
    current <- normalize(current)
    fit <- trendVar(current, parametric=TRUE, use.spikes=FALSE) 
    dec <- decomposeVar(current, fit)
    plot(dec$mean, dec$total, xlab="Mean log-expression",
        ylab="Variance of log-expression", pch=16, main=x)
    curve(fit$trend(x), col="dodgerblue", add=TRUE)
    collected[[x]] <- dec
}

In [ ]:
dec.p2449 <- do.call(combineVar, collected)
dec.p2449$gene_symbol <- rowData(sce.p2449)$gene_symbol
dec.p2449 <- dec.p2449[order(dec.p2449$bio, decreasing=TRUE),]
head(dec.p2449)

### P2458

In [ ]:
labels <- sort(unique(sce.p2458$label))
par(mfrow=c(ceiling(length(labels)/2), 2), 
    mar=c(4.1, 4.1, 2.1, 0.1))
collected <- list()
for (x in unique(sce.p2458$label)) {
    current <- sce.p2458[,sce.p2458$label==x]
    if (ncol(current)<2L) { next }
    current <- normalize(current)
    fit <- trendVar(current, parametric=TRUE, use.spikes=FALSE) 
    dec <- decomposeVar(current, fit)
    plot(dec$mean, dec$total, xlab="Mean log-expression",
        ylab="Variance of log-expression", pch=16, main=x)
    curve(fit$trend(x), col="dodgerblue", add=TRUE)
    collected[[x]] <- dec
}

In [ ]:
dec.p2458 <- do.call(combineVar, collected)
dec.p2458$gene_symbol <- rowData(sce.p2458)$gene_symbol
dec.p2458 <- dec.p2458[order(dec.p2458$bio, decreasing=TRUE),]
head(dec.p2458)

## Feature selection across batches
This part of the analysis is to select which genes we are going to use to do the batch correction. We select the top-varying genes that are common between the two batches to do the benchmark.

In [ ]:
top.p2449 <- rownames(dec.p2449)[seq_len(1000)]
top.p2458 <- rownames(dec.p2458)[seq_len(1000)]
chosen <- Reduce(intersect, list(top.p2449, top.p2458))

## Correct batch effect

We are now ready to do the batch correction. We'll select the values we want to use to do the batch corrections, and we apply the `mnnCorrect` function that is the subject of the Nat Biotech publication.

In [ ]:
original <- list(logcounts(normalized.p2449[chosen,]),
                 logcounts(normalized.p2458[chosen,]))
corrected <- do.call(mnnCorrect, c(original, list(k=20, sigma=0.1)))

Now we can create a new `SingleCellExperiment` object and stick to it all the information we have from this point.

In [ ]:
omat <- do.call(cbind, original)
mat <- do.call(cbind, corrected$corrected)
colnames(mat) <- NULL
sce <- SingleCellExperiment(list(original=omat, corrected=mat))
colData(sce)$Batch <- rep(c("p2449", "p2458"),
                          lapply(corrected$corrected, ncol))
colData(sce)$label <- c(as.character(sce.p2449$label), as.character(sce.p2458$label))

Now we can do the clustering to figure out whether there is any effect of the gene expression patterns on the expression of the GFP reporter.

In [ ]:
osce <- runTSNE(sce, exprs_values="original", rand_seed=100, perplexity=5)
ot <- plotTSNE(osce, colour_by="Batch") + ggtitle("Original")
csce <- runTSNE(sce, exprs_values="corrected", rand_seed=100, perplexity=5)
ct <- plotTSNE(csce, colour_by="Batch") + ggtitle("Corrected")
options(repr.plot.width = 10, repr.plot.height = 4)
multiplot(ot, ct, cols=2)

In [ ]:
ct.jurkat <- plotTSNE(csce, by_exprs_values="corrected", colour_by="label",
                     size_by="FILIONG01")
options(repr.plot.width = 5, repr.plot.height = 4)
plot(ct.jurkat)

The plot above can be considered to be our semi-final result. There is little evidence that the cells that have different expression values for the GFP reporters cluster together based on the gene expression patterns.

We can check that this result is robust by performing the clustering only with the treated cells.

In [ ]:
treated.sce.clustered <- runTSNE(csce[,csce$label=="J-Lat+SAHA"],
                                 exprs_values="corrected", rand_seed=100, perplexity=5)
plotTSNE(treated.sce.clustered, by_exprs_values="corrected", colour_by="FILIONG01")

In this plot as well we do not see any evidence for a specific pattern.

## Cell cycle

The next step is to take the cell cycle into account. The `cyclone` function from the `scran` package is designed to do the assignment in a semi-supervised manner.

The assignment is based on measuring the expression levels of pairs of genes. The list can be supplied if we know cells that have a certain cell cycle stage. In the case of this data set there is no such information. Therefore the package supplies us with a precompiled list of pairs of genes.

In [ ]:
# load the list of pairs of genes
hs.pairs <- readRDS(system.file("exdata", "human_cycle_markers.rds", package="scran"))

Before we can proceed with the assignment we face one problem: the list of genes contained in the pairs list is in the standard Ensembl format, whereas we have a more detailed information on the splicing variant. We need to fix such problem. A brutal approach is to just eliminate the genes that give rise to duplicates in the list (45 out of more than 58000, shouldn't be a big deal).

In [ ]:
gene_short_names <- gsub("\\..*", "", rownames(sce.p2449))
non_duplicated <- !duplicated(gene_short_names)
mygenes <- rownames(sce.p2449[non_duplicated])

# P2449
newmat.p2449 <- counts(sce.p2449)[mygenes,]
newfdata.p2449 <- rowData(sce.p2449)$gene_symbol[non_duplicated]
sce.p2449.new <- SingleCellExperiment(list(counts=newmat.p2449), rowData=newfdata.p2449)
rownames(sce.p2449.new) <- gene_short_names[non_duplicated]

# P2458
newmat.p2458 <- counts(sce.p2458)[mygenes,]
newfdata.p2458 <- rowData(sce.p2458)$gene_symbol[non_duplicated]
sce.p2458.new <- SingleCellExperiment(list(counts=newmat.p2458), rowData=newfdata.p2458)
rownames(sce.p2458.new) <- gene_short_names[non_duplicated]

Now we're ready to assign the phases to each cell.

In [ ]:
assignments.p2449 <- cyclone(sce.p2449.new, hs.pairs)
assignments.p2458 <- cyclone(sce.p2458.new, hs.pairs)

Let's see how things look like: how does the expression relate to the cell cycle phase?

In [ ]:
colData(sce)$phase <- c(assignments.p2449$phases, assignments.p2458$phases)
jlat <- colData(sce)$label == "J-Lat+SAHA"

In [ ]:
options(repr.plot.width = 5, repr.plot.height = 4)
corrected.hist.plot <- 
      ggplot(data.frame(expr=assays(sce)$corrected["FILIONG01",jlat],
                        phase=colData(sce)$phase[jlat]),
             aes(x=phase, y=expr)) + 
      geom_boxplot() +
      xlab("Phase") + ylab('Normalized GFP')
original.hist.plot <- 
      ggplot(data.frame(expr=assays(sce)$original["FILIONG01",jlat],
                        phase=colData(sce)$phase[jlat]),
             aes(x=phase, y=expr)) + 
      geom_boxplot() +
      xlab("Phase") + ylab('Original GFP')
multiplot(corrected.hist.plot, original.hist.plot, cols=2)